# Isprobavanje Mask R-CNN-a


In [1]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-aadc7qfw
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-aadc7qfw
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264362 sha256=67ac3439a2d712c9a5be5bf7450867999e48993a1f6cb5da8248ab27bc095c99
  Stored in directory: /tmp/pip-ephem-wheel-cache-qcahh00n/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [2]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 116845, done.
remote: Counting objects: 100% (9525/9525), done.
remote: Compressing objects: 100% (858/858), done.
remote: Total 116845 (delta 8740), reused 9319 (delta 8619), pack-reused 107320
Receiving objects: 100% (116845/116845), 226.75 MiB | 27.81 MiB/s, done.
Resolving deltas: 100% (101413/101413), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from engine import train_one_epoch, evaluate
import utils
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import time
import os
import copy
import transforms as T

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import time
from datetime import datetime
import random

plt.ion()   # interactive mode

In [4]:
# Access to uploaded files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dir_prefix = 'drive/My Drive/Colab Notebooks/Diplomski'

In [19]:
attrs_of_interest = ['bridge', 'endofilling', 'filling', 'crown']

def default_labeler(row):
  """ returns a list of labels which ranging from 1 to n, refering to a string label in array attrs_of_interest """
  label = []
  for i in range(len(attrs_of_interest)):
    if row[attrs_of_interest[i]] == 'yes': label.append(i + 1)
  return label

def any_intervention_labeler(row):
  """ Returns [1] indicating an existance of a tooth intervention, returns [] if no intervention is found """
  for i in range(len(attrs_of_interest)):
    if row[attrs_of_interest[i]] == 'yes': return [1]
  return []

def get_single_intervention_labeler(attr):
  def single_intervention_labeler(row):
    if row[attr] == 'yes': return [1]
    return []
  return single_intervention_labeler


class XRayDataset(torch.utils.data.Dataset):
    def __init__(self, root, csv_file, subset=None, transforms=None, labeler=default_labeler):
        self.root = root
        self.transforms = transforms
        self.csv_file = csv_file
        self.subset = subset
        self.labeler = labeler
        self.df = pd.read_csv(csv_file, sep = ";")
        if self.subset: self.df = self.subset(self.df)
        self.object_data_by_img = dict()
        
        for index, row in self.df.iterrows():       # Returns integer for a label if [] is returned the row is skipped 
          label = self.labeler(row)
          for lbl in label:
            if not row['img_name'] in self.object_data_by_img: 
              self.object_data_by_img[row['img_name']] = list()
            
            self.object_data_by_img[row['img_name']].append((lbl, row['x1'], row['y1'], row['x2'], row['y2']))

        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(self.object_data_by_img))
    
    def remove_indices(self, indices):
      imgs_to_remove = []
      for i in indices:
        imgs_to_remove.append(self.imgs[i])
      
      for curr_img in imgs_to_remove:
        self.imgs.remove(curr_img)
        del self.object_data_by_img[curr_img]
    
    def relabel(self, relabeler):
      self.labeler = relabeler
      self.object_data_by_img = dict()
        
      for index, row in self.df.iterrows():       # Returns integer for a label if [] is returned the row is skipped 
        label = self.labeler(row)
        for lbl in label:
          if not row['img_name'] in self.object_data_by_img: 
            self.object_data_by_img[row['img_name']] = list()
          
          self.object_data_by_img[row['img_name']].append((lbl, row['x1'], row['y1'], row['x2'], row['y2']))

      self.imgs = list(sorted(self.object_data_by_img))


    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")

        # get bounding box coordinates for each object
        objs = self.object_data_by_img[self.imgs[idx]]
        num_objs = len(objs)
        labels = [x[0] for x in objs]
        boxes = [[x[1], x[2], x[3], x[4]] for x in objs] # Get x1, y1, x2, y2, bounding box coordinates of the tooth

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # convert everything into a torch.Tensor
        labels = torch.as_tensor(labels, dtype=torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [7]:
# Func for more transforms if the need arises
def get_transform():
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [8]:
def keep_only_subset(df, num_of_total_negatives=-1, num_of_positives=-1):
  df_some_yes = df[~((df['crown'] == 'no') & (df['endofilling'] == 'no') & (df['filling'] == 'no') & (df['bridge'] == 'no'))]
  df_all_no = df[(df['crown'] == 'no') & (df['endofilling'] == 'no') & (df['filling'] == 'no') & (df['bridge'] == 'no')]
  if num_of_total_negatives >= 0:
    df_all_no = df_all_no.sample(n=num_of_total_negatives, random_state=1)
  if num_of_positives >= 0:
    df_some_yes = df_some_yes.sample(n=num_of_positives, random_state=1)
  print(f'len(df_some_yes): {len(df_some_yes)}, len(df_all_no): {len(df_all_no)}')
  return pd.concat([df_some_yes,df_all_no]).copy()

In [9]:
img_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
image_datasets = dict()
dataloaders = dict()
dataset_sizes = dict()
class_names = dict()

csv_filename = os.path.join(dir_prefix, 'multi_label_dataset', 'tooth_info.csv')
img_dir = os.path.join(dir_prefix, 'data_store')
dataset = XRayDataset(root=img_dir,
                      csv_file=csv_filename,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      labeler=any_intervention_labeler)


first = dataset[0]
print(first[1])
# total_number = len(dataset)
# split_size = [int(total_number * 0.7), total_number - int(total_number * 0.7)]
# dataset_train, dataset_val = random_split(dataset, split_size, generator=torch.Generator().manual_seed(42))

# image_datasets = {'train': dataset_train, 'val': dataset_val}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = attrs_of_interest

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

len(df_some_yes): 8790, len(df_all_no): 0
{'boxes': tensor([[ 789.,  421.,  942.,  767.],
        [ 902.,  407., 1099.,  774.],
        [1486.,  404., 1609.,  811.],
        [2069.,  444., 2262.,  771.],
        [1539.,  801., 1629., 1137.],
        [1469.,  804., 1576., 1147.]]), 'labels': tensor([1, 1, 1, 1, 1, 1]), 'image_id': tensor([0]), 'area': tensor([52938., 72299., 50061., 63111., 30240., 36701.]), 'iscrowd': tensor([0, 0, 0, 0, 0, 0])}


In [10]:
print(dataset.imgs[0])
print(len(dataset))

abaft-real-heady-goofy-gruesome-iron_22ff963593dc2d3e0310315f5bdf7cd8.jpg
932


In [ ]:
# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 0 background class + 1 class for generic tooth intervention
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [11]:
import math
import sys
import time

import torch
import torchvision.models.detection.mask_rcnn
import utils
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset

@torch.inference_mode()
def evaluate_specific(model, data_loader, device, labels):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = ["bbox"]
    coco_evaluator = CocoEvaluator(coco, iou_types)
    coco_evaluator.coco_eval["bbox"].params.catIds = labels # labels should be a list of classes ex. [1, 3] to analyze
    coco_evaluator.coco_eval["bbox"].params.iouThrs = [.5]

    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [12]:
def get_train_test_datasets(len_train):
  """ Filter is used after randperm indices were split into test and train lists of length specified by len_train """
  torch.manual_seed(1)
  dataset = XRayDataset(root=img_dir,
                      csv_file=csv_filename,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform(),
                      labeler=any_intervention_labeler)
  dataset_test = XRayDataset(root=img_dir,
                      csv_file=csv_filename,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform(),
                      labeler=any_intervention_labeler)

  # split the dataset in train and test set
  indices = torch.randperm(len(dataset)).tolist()
  train_idx = indices[:-len_train]
  test_idx = indices[-len_train:]
  dataset = torch.utils.data.Subset(dataset, train_idx)
  dataset_test = torch.utils.data.Subset(dataset_test, test_idx)
  return dataset, dataset_test




In [ ]:
def main(model):
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # use our dataset and defined transformations
    dataset, dataset_test = get_train_test_datasets(len_train=186)

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [ ]:
main(model)

len(df_some_yes): 8790, len(df_all_no): 0
len(df_some_yes): 8790, len(df_all_no): 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [  0/373]  eta: 0:24:42  lr: 0.000018  loss: 2.6758 (2.6758)  loss_classifier: 0.8279 (0.8279)  loss_box_reg: 0.3724 (0.3724)  loss_objectness: 1.4098 (1.4098)  loss_rpn_box_reg: 0.0658 (0.0658)  time: 3.9751  data: 2.3091  max mem: 2210
Epoch: [0]  [ 10/373]  eta: 0:09:42  lr: 0.000153  loss: 2.0328 (2.0790)  loss_classifier: 0.7401 (0.7261)  loss_box_reg: 0.3829 (0.4379)  loss_objectness: 0.6257 (0.8394)  loss_rpn_box_reg: 0.0768 (0.0755)  time: 1.6041  data: 0.2419  max mem: 2473
Epoch: [0]  [ 20/373]  eta: 0:08:44  lr: 0.000287  loss: 1.3321 (1.6124)  loss_classifier: 0.5348 (0.5784)  loss_box_reg: 0.4635 (0.4683)  loss_objectness: 0.1596 (0.5126)  loss_rpn_box_reg: 0.0348 (0.0531)  time: 1.3621  data: 0.0344  max mem: 2473
Epoch: [0]  [ 30/373]  eta: 0:08:15  lr: 0.000421  loss: 1.1317 (1.4479)  loss_classifier: 0.4124 (0.5183)  loss_box_reg: 0.5715 (0.4968)  loss_objectness: 0.1498 (0.3856)  loss_rpn_box_reg: 0.0312 (0.0472)  time: 1.3590  data: 0.0343  max mem: 2473


In [ ]:
# datetime object containing current date and time
now = datetime.now()
 
# dd/mm/YY H:M:S
dt_string = now.strftime("%d:%m:%Y-%H:%M:%S")

torch.save(model, os.path.join(dir_prefix, 'trained_models', f'2class-faster-rcnn_model-{dt_string}-epochs-10'))

In [13]:
# Load the model 

model_ld = torch.load(os.path.join(dir_prefix, 'trained_models', '2class-faster-rcnn_model-05:04:2022-20:28:25-epochs-10'))
model_ld.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [14]:
#model_ld = model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# use our dataset and defined transformations
_, dataset_test = get_train_test_datasets(len_train=186)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

print("\n\nEVALUATION AT IOU=0.5:")
for i in [1]:
  evaluate_specific(model_ld, data_loader_test, device, [i])
  print()

len(df_some_yes): 8790, len(df_all_no): 0
len(df_some_yes): 8790, len(df_all_no): 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))




EVALUATION AT IOU=0.5:
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Test:  [  0/186]  eta: 0:05:20  model_time: 0.6765 (0.6765)  evaluator_time: 0.0204 (0.0204)  time: 1.7254  data: 0.9845  max mem: 556
Test:  [100/186]  eta: 0:00:31  model_time: 0.3161 (0.3209)  evaluator_time: 0.0020 (0.0023)  time: 0.3500  data: 0.0175  max mem: 557
Test:  [185/186]  eta: 0:00:00  model_time: 0.3161 (0.3184)  evaluator_time: 0.0018 (0.0022)  time: 0.3462  data: 0.0174  max mem: 674
Test: Total time: 0:01:06 (0.3595 s / it)
Averaged stats: model_time: 0.3161 (0.3184)  evaluator_time: 0.0018 (0.0022)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.50 | area=   all | maxDets=100 ] = 0.927
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.50 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.50 | area=medium | maxDets=100 ] = 0.607
 Aver

IndexError: ignored

In [21]:
print("\n\nRECALL EVALUATION AT IOU=0.5")
for i, attr in enumerate(attrs_of_interest):
  print()
  print()
  print(f"{attr} evaluation subset:")
  torch.manual_seed(1)
  dataset_test = XRayDataset(root=img_dir,
                      csv_file=csv_filename,
                      subset=lambda df: keep_only_subset(df, num_of_total_negatives=0),
                      transforms=get_transform(),
                      labeler=any_intervention_labeler)
  indices = torch.randperm(len(dataset)).tolist()
  train_idx = indices[:-186]
  test_idx = indices[-186:]
  dataset_test.remove_indices(train_idx)
  dataset_test.relabel(get_single_intervention_labeler(attr))
  data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
  evaluate_specific(model_ld, data_loader_test, device, [1])



RECALL EVALUATION AT IOU=0.5


bridge evaluation subset:
len(df_some_yes): 8790, len(df_all_no): 0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


creating index...
index created!
Test:  [  0/223]  eta: 0:05:41  model_time: 0.4343 (0.4343)  evaluator_time: 0.0112 (0.0112)  time: 1.5304  data: 1.0649  max mem: 674
Test:  [100/223]  eta: 0:00:44  model_time: 0.3150 (0.3161)  evaluator_time: 0.0013 (0.0016)  time: 0.3449  data: 0.0157  max mem: 674
Test:  [200/223]  eta: 0:00:08  model_time: 0.3127 (0.3163)  evaluator_time: 0.0014 (0.0017)  time: 0.3452  data: 0.0166  max mem: 674
Test:  [222/223]  eta: 0:00:00  model_time: 0.3146 (0.3161)  evaluator_time: 0.0014 (0.0017)  time: 0.3435  data: 0.0162  max mem: 674
Test: Total time: 0:01:18 (0.3542 s / it)
Averaged stats: model_time: 0.3146 (0.3161)  evaluator_time: 0.0014 (0.0017)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.50 | area=   all | maxDets=100 ] = 0.139
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.00

TypeError: ignored

In [22]:
font = ImageFont.truetype("LiberationMono-Bold.ttf", size=60)
font2 = ImageFont.truetype("LiberationMono-Bold.ttf", size=80)

In [ ]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

_, dataset_test = get_train_test_datasets(len_train=186)
n = 5
# pick n images from the test set
indexes = random.sample(range(len(dataset_test)), n)

for i in indexes:
  img, t = dataset_test[i]
  # put the model in evaluation mode
  model_ld.eval()
  with torch.no_grad():
      prediction = model_ld([img.to(device)])
    
  prediction = prediction[0]
  print("Prediction:")
  print(prediction)
  print("Target:")
  print(t)

  img_pred = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
  imgd = ImageDraw.Draw(img_pred)
  for i, s in enumerate(prediction['scores']):
    if s < 0.51: continue
    imgd.rectangle(prediction['boxes'][i].tolist(), outline ="blue", width=4)
    imgd.text(prediction['boxes'][i].tolist()[:2], str(int(prediction['labels'][i])), font=font, align ="left")
  imgd.text((20,20), 'Prediction', font=font2, align ="left", fill='blue')
  display(img_pred)

  img_target = Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())
  imgdt = ImageDraw.Draw(img_target)  
  font = ImageFont.truetype("LiberationMono-Bold.ttf", size=60)
  already_seen = dict()
  for i, l in enumerate(t['labels']):
    imgdt.rectangle(t['boxes'][i].tolist(), outline ="blue", width=4)
    txt_coords = tuple(t['boxes'][i].tolist()[:2])
    shift = already_seen.get(txt_coords, 0) * 45
    final_coords = (txt_coords[0] + shift, txt_coords[1])
    imgdt.text(final_coords, str(int(l)), font=font, align ="left")
    already_seen[txt_coords] = already_seen.get(txt_coords, 0) + 1
  imgdt.text((20,20), 'Target', font=font2, align ="left", fill='blue')
  display(img_target)


len(df_some_yes): 8790, len(df_all_no): 0
len(df_some_yes): 8790, len(df_all_no): 0
Prediction:
{'boxes': tensor([[2030.5542,  728.0590, 2225.4292, 1055.7222],
        [1048.7112,  762.3846, 1237.2052, 1099.9443],
        [2107.9231,  413.7835, 2303.4995,  727.7932],
        [1048.7977,  461.7788, 1220.4082,  767.1240],
        [1209.5386,  786.4845, 1337.9055, 1140.8593],
        [2258.2942,  686.0786, 2538.0527,  937.9734],
        [ 937.0775,  465.7199, 1088.1486,  760.2395],
        [1268.1117,  417.4923, 1385.7300,  776.7562],
        [ 894.5991,  752.0772, 1095.1034, 1065.2474],
        [2250.2764,  414.9732, 2400.3418,  716.0773],
        [ 827.4200,  470.9974,  962.4047,  733.7302],
        [ 741.4439,  717.6883,  965.1513,  945.1351],
        [1977.6904,  383.6466, 2086.3394,  740.7591],
        [1324.0504,  359.0901, 1449.6134,  765.0036]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), 'scores': tensor([0.9968, 0.9957, 0.9949

In [ ]:
import matplotlib
system_fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

In [ ]:
system_fonts

['/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf',
 '/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf',
 '/usr/share/fonts/truetype/liberat